<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=befdadad9a667d92bb77fb441502c7991ed3fad5649382bdb2ec3645937af1bc
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-09 15:07:10
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  4.95 L
Current:  1.39 C
-------------------
Today PnL: 23.59 K (0.17%)
Current PnL: -29.08 L (-18.79%)
CY Booked + Current PnL: -14.79 L (-9.56%)
-------------------
Total profit:  1.14 L
Total loss:  -30.23 L
-------------------
Total Booked + Current PnL: 12.19 L (9.53%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.27%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.33 C
Est FTT PnL: 93.71 L (67.33%)
Deployed:  1.28 C
Current:  1.39 C
CAGR/XIRR %: 4.79%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,-0.34,-17.92,21.95,0.09,18156.0,-18064.0,82713.0,91.89,32.0,M-SC,2.19,253.0,-0.99,0.62,8.67,OX40N,NTT,DURABLES
50,MASFIN,398.61,1.99,-2.86,25.65,22.05,24412.0,-2805.0,95175.0,-16.44,54.0,H-SC,7.04,164.0,-0.11,0.71,38.34,XR,ATH,FINANCE
43,ITC,452.00,-0.21,-1.91,12.59,10.44,24715.0,-3829.0,196309.0,-41.56,43.0,X-LC,1.66,5.0,-0.15,1.46,3.54,X40,NTT,FMCG
39,INDIGOPNTS,1408.00,-0.81,-15.18,17.91,0.02,26519.0,-26491.0,148068.0,126.00,45.0,M-SC,16.71,234.0,-1.00,1.10,28.79,OX40N,NTT,PAINTS
35,ICICIGI,2252.93,-0.82,3.77,15.83,20.19,28634.0,6565.0,180885.0,-18.61,40.0,X-MC,5.97,68.0,0.23,1.35,19.45,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,RELAXO,1176.00,6.18,-45.29,178.38,52.31,141671.0,-65739.0,79421.0,-41.73,54.0,X-SC,5.88,91.0,-0.46,0.59,6.31,X40N,NTT,FOOTWEAR
61,ROUTE,2227.21,5.33,-47.55,222.32,69.05,155159.0,-63275.0,69791.0,-57.43,50.0,H-SC,23.11,140.0,-0.41,0.52,7.63,SR,ATH,IT
11,BATAINDIA,2096.00,4.80,-35.88,110.85,35.20,91458.0,-46164.0,82506.0,8.16,43.0,X-SC,15.12,92.0,-0.50,0.61,4.80,X40,NTT,FOOTWEAR
56,RAJESHEXPO,518.00,4.19,-63.55,174.51,0.07,87265.0,-87171.0,50006.0,1733.33,53.0,L-SC,2.35,268.0,-1.00,0.37,24.37,OX40N,NTT,JEWELLERY
10,BANDHANBNK,400.00,4.06,-32.19,183.75,92.42,347104.0,-89659.0,188900.0,32.90,34.0,H-SC,7.85,170.0,-0.26,1.41,9.17,XY24,NTT,BANKS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,ASIANPAINT,3460.25,-4.41,-2.16,23.62,20.95,58181.0,-5447.0,246321.0,-3.16,49.0,X-LC,19.02,31.0,-0.09,1.83,32.70,X40,ATH,PAINTS
49,LAOPALA,464.00,-3.36,-40.15,132.41,39.09,103892.0,-52643.0,78462.0,71.30,23.0,H-SC,7.43,155.0,-0.51,0.58,1.05,AR,NTT,CERAMICS
28,GREENPANEL,537.00,-2.25,-39.38,136.12,43.14,151074.0,-72092.0,110986.0,148.71,22.0,M-SC,8.66,240.0,-0.48,0.83,4.40,XY24,NTT,MISC
23,DMART,5391.45,-2.14,-8.09,39.40,28.12,76192.0,-17027.0,193380.0,-20.72,30.0,X-LC,4.10,19.0,-0.22,1.44,13.62,X40N,ATH,FMCG
2,ABBOTINDIA,35195.00,-2.12,-6.19,25.18,17.44,35397.0,-9272.0,140575.0,-19.22,32.0,X-MC,4.57,63.0,-0.26,1.05,11.78,X40,ATH,PHARMA


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,1.37,-7.92,125.80,107.91,171024.0,-11695.0,135949.0,-25.01,32.0,M-SC,11.87,216.0,-0.07,1.01,1.37,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-0.81,-15.18,17.91,0.02,26519.0,-26491.0,148068.0,126.00,45.0,M-SC,16.71,234.0,-1.00,1.10,28.79,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-0.34,-17.92,21.95,0.09,18156.0,-18064.0,82713.0,91.89,32.0,M-SC,2.19,253.0,-0.99,0.62,8.67,OX40N,NTT,DURABLES
66,SIS,528.00,0.61,-24.72,61.07,21.26,51048.0,-27443.0,83589.0,1971.50,46.0,H-SC,4.59,166.0,-0.54,0.62,12.94,OX40N,NTT,MISC
47,KANSAINER,340.00,0.12,-25.87,53.07,13.47,106091.0,-69759.0,199908.0,-69.48,26.0,H-SC,3.21,158.0,-0.66,1.49,1.65,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,-1.46,5.64,63.06,72.27,112884.0,9563.0,179011.0,-9.26,64.0,M-LC,2.23,99.0,0.08,1.33,12.17,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-1.46,5.64,63.06,72.27,112884.0,9563.0,179011.0,-9.26,64.0,M-LC,2.23,99.0,0.08,1.33,12.17,XR,NTT,IT
38,INDIAMART,4810.62,-0.54,-2.83,112.75,106.72,135124.0,-3492.0,119844.0,-51.96,34.0,H-SC,1.19,139.0,-0.03,0.89,19.88,AR,ATH,MISC
25,FINCABLES,1641.55,1.37,-7.92,125.80,107.91,171024.0,-11695.0,135949.0,-25.01,32.0,M-SC,11.87,216.0,-0.07,1.01,1.37,OX40N,ATH,CABLES
85,ZYDUSLIFE,1286.17,-0.51,-2.99,39.82,35.63,80953.0,-6276.0,203298.0,-16.74,35.0,H-MC,3.16,119.0,-0.08,1.51,13.18,AR,ATH,PHARMA
37,IEX,219.00,-0.47,-4.14,55.04,48.63,105498.0,-8278.0,191676.0,-36.24,48.0,H-SC,14.08,136.0,-0.08,1.43,6.93,XR,NTT,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,1.98,-21.89,90.05,48.45,141036.0,-43886.0,156620.0,-27.60,21.0,X-MC,11.17,64.0,-0.31,1.17,1.98,X40N,ATH,FINANCE
22,DIXON,18931.72,-0.46,-13.30,40.23,21.58,48879.0,-18639.0,121500.0,-56.26,24.0,X-MC,5.46,56.0,-0.38,0.91,7.57,X40N,ATH,IT
75,TMPV,600.00,-1.50,-17.46,72.27,42.18,164874.0,-48274.0,228136.0,-25.52,27.0,X-LC,4.53,3.0,-0.29,1.70,0.00,XY24,NTT,AUTO
33,HINDUNILVR,2922.00,-0.30,-10.25,26.65,13.67,61484.0,-26353.0,230710.0,-16.94,29.0,X-LC,2.98,14.0,-0.43,1.72,8.92,XY25,NTT,FMCG
53,PAGEIND,51605.08,-0.04,-9.08,38.65,26.07,57542.0,-14860.0,148880.0,-33.02,30.0,X-MC,11.48,55.0,-0.26,1.11,0.00,X40,ATH,APPARELS


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,AWL,485.0,-1.60,-21.95,96.67,53.50,227661.0,-66248.0,235503.0,-62.23,31.0,X-MC,1.25,58.0,-0.29,1.75,4.60,XY24,NTT,FMCG
43,ITC,452.0,-0.21,-1.91,12.59,10.44,24715.0,-3829.0,196309.0,-41.56,43.0,X-LC,1.66,5.0,-0.15,1.46,3.54,X40,NTT,FMCG
3,ACC,3906.0,-0.64,-25.01,118.88,64.15,212141.0,-59501.0,178450.0,-56.31,35.0,X-SC,1.79,82.0,-0.28,1.33,0.10,XY24,BTT,CEMENT
15,CAMS,4762.0,-0.48,-80.35,535.10,24.79,264805.0,-202369.0,49487.0,-81.32,40.0,X-SC,1.85,86.0,-0.76,0.37,20.98,X40N,NTT,MISC
36,ICICIPRULI,790.0,1.27,3.46,26.59,30.97,48619.0,6109.0,182847.0,-20.92,55.0,X-MC,2.06,75.0,0.13,1.36,16.45,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,PAGEIND,51605.08,-0.04,-9.08,38.65,26.07,57542.0,-14860.0,148880.0,-33.02,30.0,X-MC,11.48,55.0,-0.26,1.11,0.00,X40,ATH,APPARELS
75,TMPV,600.00,-1.50,-17.46,72.27,42.18,164874.0,-48274.0,228136.0,-25.52,27.0,X-LC,4.53,3.0,-0.29,1.70,0.00,XY24,NTT,AUTO
54,PGHH,17907.65,-1.00,-5.89,42.14,33.76,79638.0,-11835.0,188985.0,-33.58,34.0,X-MC,5.09,57.0,-0.15,1.41,0.10,X40,ATH,FMCG
3,ACC,3906.00,-0.64,-25.01,118.88,64.15,212141.0,-59501.0,178450.0,-56.31,35.0,X-SC,1.79,82.0,-0.28,1.33,0.10,XY24,BTT,CEMENT
31,HAVELLS,2069.16,0.92,-9.06,45.68,32.48,136247.0,-29728.0,298263.0,-13.63,41.0,X-MC,5.17,67.0,-0.22,2.22,1.33,X40,ATH,ELECTRICAL


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,1.49,-30.01,104.09,42.84,47358.0,-19509.0,45497.0,-54.27,41.0,X-SC,37.02,83.0,-0.41,0.34,2.93,XY24,NTT,MISC
15,CAMS,4762.00,-0.48,-80.35,535.10,24.79,264805.0,-202369.0,49487.0,-81.32,40.0,X-SC,1.85,86.0,-0.76,0.37,20.98,X40N,NTT,MISC
59,RELAXO,1176.00,6.18,-45.29,178.38,52.31,141671.0,-65739.0,79421.0,-41.73,54.0,X-SC,5.88,91.0,-0.46,0.59,6.31,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,4.80,-35.88,110.85,35.20,91458.0,-46164.0,82506.0,8.16,43.0,X-SC,15.12,92.0,-0.50,0.61,4.80,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-0.46,-13.30,40.23,21.58,48879.0,-18639.0,121500.0,-56.26,24.0,X-MC,5.46,56.0,-0.38,0.91,7.57,X40N,ATH,IT


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-0.85,-12.81,36.80,19.28,111006.0,-44310.0,301646.0,-23.95,63.0,X-LC,5.77,1.0,-0.40,2.25,11.51,X40,ATH,IT
41,INFY,2275.00,-0.77,8.81,42.33,54.87,144116.0,27571.0,340459.0,-15.29,64.0,X-LC,3.38,2.0,0.19,2.54,17.82,X40,BTT,IT
75,TMPV,600.00,-1.50,-17.46,72.27,42.18,164874.0,-48274.0,228136.0,-25.52,27.0,X-LC,4.53,3.0,-0.29,1.70,0.00,XY24,NTT,AUTO
81,VBL,671.64,0.44,-4.69,42.34,35.67,127460.0,-14802.0,301040.0,-16.08,53.0,X-LC,4.81,4.0,-0.12,2.24,8.52,X40N,ATH,FMCG
43,ITC,452.00,-0.21,-1.91,12.59,10.44,24715.0,-3829.0,196309.0,-41.56,43.0,X-LC,1.66,5.0,-0.15,1.46,3.54,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,0.35,-37.66,113.18,32.89,54471.0,-29078.0,48128.0,-694.37,54.0,L-MC,6.16,259.0,-0.53,0.36,32.54,XR,NTT,BANKS
6,ASIANTILES,137.00,0.38,-19.15,124.70,81.67,94581.0,-17963.0,75847.0,6811.11,33.0,L-SC,14.90,271.0,-0.19,0.56,46.88,XR,NTT,CERAMICS
50,MASFIN,398.61,1.99,-2.86,25.65,22.05,24412.0,-2805.0,95175.0,-16.44,54.0,H-SC,7.04,164.0,-0.11,0.71,38.34,XR,ATH,FINANCE
13,BSOFT,831.70,0.54,-21.96,93.22,50.79,102323.0,-30888.0,109765.0,0.12,70.0,H-SC,5.84,151.0,-0.30,0.82,27.80,XR,ATH,IT
39,INDIGOPNTS,1408.00,-0.81,-15.18,17.91,0.02,26519.0,-26491.0,148068.0,126.00,45.0,M-SC,16.71,234.0,-1.00,1.10,28.79,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,0.54,-21.96,93.22,50.79,102323.0,-30888.0,109765.0,0.12,70.0,H-SC,5.84,151.0,-0.30,0.82,27.80,XR,ATH,IT
32,HCLTECH,1922.01,-1.86,7.55,15.98,24.73,41577.0,18256.0,260180.0,12.34,62.0,X-LC,5.74,7.0,0.44,1.94,23.93,X40,ATH,IT
84,WIPRO,420.00,-1.46,5.64,63.06,72.27,112884.0,9563.0,179011.0,-9.26,64.0,M-LC,2.23,99.0,0.08,1.33,12.17,XR,NTT,IT
56,RAJESHEXPO,518.00,4.19,-63.55,174.51,0.07,87265.0,-87171.0,50006.0,1733.33,53.0,L-SC,2.35,268.0,-1.00,0.37,24.37,OX40N,NTT,JEWELLERY
73,TCS,4389.97,-0.85,-12.81,36.80,19.28,111006.0,-44310.0,301646.0,-23.95,63.0,X-LC,5.77,1.0,-0.40,2.25,11.51,X40,ATH,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.81
1,25,45.72
2,50,76.63


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.99
MC    30.31
LC    24.67
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.28
X40      22.52
X40N     10.93
XR       10.05
XY25      9.39
AR        9.30
OX40N     7.71
SR        1.04
MH        0.75
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.48
X-MC    23.57
X-LC    21.12
M-SC    12.07
X-SC     7.05
H-MC     4.97
M-MC     1.41
L-SC     1.39
M-LC     1.33
H-LC     1.17
L-LC     1.05
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.43,-7.51,42.28
IT,12.85,-16.89,80.08
FINANCE,9.51,-21.45,72.77
ELECTRICAL,6.24,-12.03,53.06
MISC,6.10,-58.60,123.57
PAINTS,6.04,-13.66,30.54
INSURANCE,4.62,-1.96,37.90
PHARMA,4.08,-1.85,34.45
AUTO,3.42,-25.18,78.85


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3239648.0,21
AR,1322124.0,10
XR,1312472.0,13
X40,1015446.0,14
X40N,982716.0,9
OX40N,754317.0,10
XY25,380450.0,6
SR,280927.0,2
MH,83341.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3690463.0,25
M-SC,1438053.0,15
X-MC,1317887.0,16
X-SC,973149.0,8
X-LC,927371.0,11
H-MC,411302.0,3
L-SC,274196.0,3
M-LC,112884.0,1
H-LC,76414.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1294194.0      6
           AR         901139.0      5
M-SC       XY24       815271.0      6
H-SC       XR         783358.0      7
X-SC       X40N       503125.0      3
X-MC       X40        458697.0      7
           XY24       397057.0      3
X-LC       X40        379595.0      5
H-SC       OX40N      347504.0      4
M-SC       OX40N      319548.0      5
X-SC       XY24       292870.0      3
H-SC       SR         280927.0      2
X-MC       X40N       275939.0      4
X-LC       XY24       245119.0      2
H-MC       AR         216165.0      2
X-LC       X40N       203652.0      2
H-MC       XY24       195137.0      1
L-SC       XR         186931.0      2
X-MC       XY25       186194.0      2
X-SC       X40        177154.0      2
M-SC       XR         174828.0      2
           AR         128406.0      2
M-LC       XR         112884.0      1
X-LC       XY25        99005.0      2
L-SC       OX40N       87265.0      1
H-SC       MH          83341.0      1
H-LC       AR          76414.0      1
M-MC       XY25        56405.0      1
L-MC       XR          54471.0      1
L-LC       XY25        38846.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 26.0 seconds
